In [1]:
#library import

from os.path import join
import math
from math import sqrt
import pandas as pd
import numpy as np
import sklearn
import scipy.stats
from scipy.sparse.linalg import svds
from sklearn.cross_validation import train_test_split
import operator
import time

/home/hope/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#read data path
BASE_PATH = "/media/sf_졸프"
FILE_NAME = "userData.txt"

watchHistory = pd.read_csv(join(BASE_PATH, FILE_NAME), sep="\t")
watchMatrix = pd.crosstab(watchHistory.VIDEO_NUM, watchHistory.USER_NUM).replace(0, np.nan)

#read data the first process
N_USERS = watchHistory.USER_NUM.unique().shape[0]
N_ITEMS = watchHistory.VIDEO_NUM.unique().shape[0]

history = np.zeros((N_USERS, N_ITEMS))

"""weight to each user**"""
#if user watch video, mark 1
for row in watchHistory.itertuples():
    history[row[1]-1, row[2]-1] = row[3]

#don't need
history_train, history_test = train_test_split(history, test_size = 0.33, random_state=42)

#predict user_i's rating if they didn't watch item.

In [3]:
#singular vector decomposition
user_ratings_mean = np.zeros(N_USERS)

start = time.time()
demeaned_history = np.zeros((N_USERS, N_ITEMS))
for i in range(N_USERS) :
    sum = 0
    count = 0
    for j in range(N_ITEMS) :
        if(history[i][j] != 0) :
            sum += history[i][j]
            count += 1
    user_ratings_mean[i] = float(sum / float(count))
    for k in range(N_ITEMS) :
        if(history[i][k] != 0) :
            demeaned_history[i][k] = history[i][k] - user_ratings_mean[i]
        else :
            demeaned_history[i][k] = history[i][k]
    
U, sigma, Vt = svds(demeaned_history, k=1)
sigma = np.diag(sigma)
for i in range(len(sigma)) :
    sigma[i][i] = math.sqrt(sigma[i][i])

predict_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1,1)

print("time of prediction by SVD : " + str(time.time() - start) + "\n")

predict_count = 0
squared_error = 0

#calculate rmse
for i in range(N_USERS):
    for j in range(N_ITEMS):
        if(history[i][j] == 0) :
            predict_count += 1
            squared_error += (predict_ratings[i][j] - history[i][j])**2

frobenius_norm = math.sqrt(squared_error)
rmse = float(frobenius_norm / float(predict_count))
print(predict_count) 
print(squared_error) 
print(frobenius_norm)
print(rmse)


time of prediction by SVD : 0.14578747749328613

575
575.0
23.979157616563597
0.04170288281141495


In [4]:
#recommend top-k video for each user